In [8]:
def run_tests(test_list):
    count = 0
    for test in test_list:
        try:
            test()
            count += 1
        except Exception as e:
            print(f"Exception: {test.__name__}: {e}")
            continue
    print(f"{count} of {len(test_list)} tests ran successfully.")

In [9]:
from test.test_component_actions_system_dispatcher import *
tests = [test_dispatcher_ev_gamestart,
         test_dispatcher_ev_gameover,
         test_dispatcher_ev_keydown]

run_tests(tests)
# --- IGNORE ---

Exception: test_dispatcher_ev_gamestart: empty range in randrange(0, 0)
Exception: test_dispatcher_ev_gameover: empty range in randrange(0, 0)
Exception: test_dispatcher_ev_keydown: empty range in randrange(0, 0)
0 of 3 tests ran successfully.


In [10]:
from test.test_component_actions_dispatchers import *
tests = [test_dispatcher_ev_quit,
         test_dispatcher_initialization]

run_tests(tests)
# --- IGNORE ---

Exception: test_dispatcher_ev_quit: empty range in randrange(0, 0)
1 of 2 tests ran successfully.


In [11]:
from test.test_game_ai import *
test_game_over_event_handling()
test_game_ai_initialization()
test_game_ai_no_events()
test_game_ai_multiple_events()
test_game_ai_event_clearing()



NameError: name 'GameState' is not defined

In [4]:
from test.test_game_map import *
test_get_map_coords()
test_in_bounds()
test_is_traversable_transparent_visible_explored()
test_out_of_bounds_checks()
test_set_visible_and_reset()
test_update_explored()
test_width_height()
test_tiles_initialization()


AttributeError: 'components.maps.room_library.CircularRoom' object has no attribute 'tile_map'

In [5]:
from test.test_roster import *
test_roster_spawn()
test_roster_entity_blocked_locations()
test_roster_entity_collision()
test_roster_live_ai_actors()
test_roster_live_actors()
test_roster_get_entity_at_location()


Failed: Roster spawn did not function as expected